In [1]:
my_packages <- c("tidyverse", "lme4", "nlme", "readxl", "lmerTest", "dplyr")

In [56]:
lapply(my_packages, require, character.only = TRUE)

Loading required package: lme4

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lme4’”
Loading required package: lmerTest

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lmerTest’”


[[1]]
[1] TRUE

[[2]]
[1] FALSE

[[3]]
[1] TRUE

[[4]]
[1] TRUE

[[5]]
[1] FALSE

[[6]]
[1] TRUE

In [6]:
# This is the start of cleaning and tidying process

In [7]:
#List excel sheets
combined_data <- excel_sheets("2_People_untidy_MD.xlsx") %>% map_df(~read_xlsx("2_People_untidy_MD.xlsx",.))
head(combined_data)

Moon,People_total,msq,Date,Time
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Muku,NA,8059.53,20220628,0
Muku,NA,8059.53,20220628,0
Muku,NA,8059.53,20220628,0
Muku,NA,8059.53,20220628,4
Muku,NA,8059.53,20220628,4
Muku,NA,8059.53,20220628,4


In [8]:
#Replace missing values with 0
combined_data[is.na(combined_data)] <- 0

In [9]:
# Divide People_total / msq to get total ppl_msq
combined_data$ppl_msq <- (combined_data$People_total / combined_data$msq)

In [10]:
#preview
head(combined_data)
tail(combined_data)

Moon,People_total,msq,Date,Time,ppl_msq
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Muku,0,8059.53,20220628,0,0
Muku,0,8059.53,20220628,0,0
Muku,0,8059.53,20220628,0,0
Muku,0,8059.53,20220628,4,0
Muku,0,8059.53,20220628,4,0
Muku,0,8059.53,20220628,4,0


Moon,People_total,msq,Date,Time,ppl_msq
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Hoku,4,8059.53,20220713,20,0.0004963069
Hoku,4,8059.53,20220713,20,0.0004963069
Hoku,4,8059.53,20220713,20,0.0004963069
Hoku,0,8059.53,20220713,24,0.0000000000
Hoku,0,8059.53,20220713,24,0.0000000000
Hoku,0,8059.53,20220713,24,0.0000000000


In [11]:
#all BP3 and OC concentration ([UVF]) data from individual CSV files; str preview showing NA values
conc_muku <- data.frame(read.csv("conc_muku.csv"))
conc_ole <- data.frame(read.csv("conc_ole.csv"))
conc_hoku <- data.frame(read.csv("conc_hoku.csv"))
str(conc_muku)
str(conc_ole)
str(conc_hoku)

'data.frame':	30 obs. of  4 variables:
 $ Moon: chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time: int  0 0 0 4 4 4 8 8 8 9 ...
 $ BP3 : num  12.3 13.9 11.1 20 20.1 ...
 $ OC  : num  NA NA NA NA NA ...
'data.frame':	30 obs. of  4 variables:
 $ Moon: chr  "ʻOlekūkolu" "ʻOlekūkolu" "ʻOlekūkolu" "ʻOlekūkolu" ...
 $ Time: int  0 0 0 4 4 4 8 8 8 9 ...
 $ BP3 : num  21.2 21.5 19.3 20.3 21.7 ...
 $ OC  : num  NA NA NA 15.6 12.8 ...
'data.frame':	30 obs. of  4 variables:
 $ Moon: chr  "Hoku" "Hoku" "Hoku" "Hoku" ...
 $ Time: int  0 0 0 4 4 4 8 8 8 9 ...
 $ BP3 : num  36.9 35.1 35.9 36.6 37.1 ...
 $ OC  : num  87.4 73.7 71 84.2 109.1 ...


In [12]:
#combine all [UVF] data to one data frame
conc <- rbind(conc_muku, conc_ole, conc_hoku)

In [13]:
#replace NA values with 0; str preview
conc[is.na(conc)] <- 0
str(conc)

'data.frame':	90 obs. of  4 variables:
 $ Moon: chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time: int  0 0 0 4 4 4 8 8 8 9 ...
 $ BP3 : num  12.3 13.9 11.1 20 20.1 ...
 $ OC  : num  0 0 0 0 0 ...


In [14]:
#combine [UVF] and pple from combined_data; head preview
conc_pple <- cbind(conc,combined_data)
head(conc_pple)

,Moon,Time,BP3,OC,Moon,People_total,msq,Date,Time,ppl_msq
,<chr>,<int>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Muku,0,12.34324,0,Muku,0,8059.53,20220628,0,0
2,Muku,0,13.92103,0,Muku,0,8059.53,20220628,0,0
3,Muku,0,11.11400,0,Muku,0,8059.53,20220628,0,0
4,Muku,4,20.03424,0,Muku,0,8059.53,20220628,4,0
5,Muku,4,20.05967,0,Muku,0,8059.53,20220628,4,0
6,Muku,4,18.89155,0,Muku,0,8059.53,20220628,4,0


In [15]:
#remove duplicate columns, reorganize columns; head preview
conc_pple_2 <- conc_pple[,-5]
conc_pple_3 <- conc_pple_2[,-8]
conc_pple_4 <- conc_pple_3[,-6]
conc_pple_fin <- conc_pple_4 %>% relocate(Date, .after = Moon)
head(conc_pple_fin)

,Moon,Date,Time,BP3,OC,People_total,ppl_msq
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,Muku,20220628,0,12.34324,0,0,0
2,Muku,20220628,0,13.92103,0,0,0
3,Muku,20220628,0,11.11400,0,0,0
4,Muku,20220628,4,20.03424,0,0,0
5,Muku,20220628,4,20.05967,0,0,0
6,Muku,20220628,4,18.89155,0,0,0


In [16]:
### Kaimana_tide dataset
Diamond_data <- read.csv("Diamond_tide.csv")
str(Diamond_data)

'data.frame':	90 obs. of  6 variables:
 $ Date    : int  20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 ...
 $ Moon    : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time    : num  0 0 0 4 4 4 8 8 8 9 ...
 $ Tide    : num  0.271 0.271 0.271 0.505 0.505 ...
 $ PreTide : num  0.24 0.24 0.24 0.54 0.54 0.54 0.54 0.54 0.54 -0.15 ...
 $ PostTide: num  0.54 0.54 0.54 -0.15 -0.15 -0.15 -0.15 -0.15 -0.15 2.28 ...


In [17]:
### Combining PreTide and PostTide columns into one
Diamond_data$PreTide_PostTide <- paste(Diamond_data$PreTide, Diamond_data$PostTide, sep="_")
str(Diamond_data)

'data.frame':	90 obs. of  7 variables:
 $ Date            : int  20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 ...
 $ Moon            : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time            : num  0 0 0 4 4 4 8 8 8 9 ...
 $ Tide            : num  0.271 0.271 0.271 0.505 0.505 ...
 $ PreTide         : num  0.24 0.24 0.24 0.54 0.54 0.54 0.54 0.54 0.54 -0.15 ...
 $ PostTide        : num  0.54 0.54 0.54 -0.15 -0.15 -0.15 -0.15 -0.15 -0.15 2.28 ...
 $ PreTide_PostTide: chr  "0.24_0.54" "0.24_0.54" "0.24_0.54" "0.54_-0.15" ...


In [18]:
### Removing old columns
##download this one as csv
tide <- Diamond_data[c("Date", "Moon", "Time", "Tide", "PreTide_PostTide")]
str(tide)

'data.frame':	90 obs. of  5 variables:
 $ Date            : int  20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 ...
 $ Moon            : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time            : num  0 0 0 4 4 4 8 8 8 9 ...
 $ Tide            : num  0.271 0.271 0.271 0.505 0.505 ...
 $ PreTide_PostTide: chr  "0.24_0.54" "0.24_0.54" "0.24_0.54" "0.54_-0.15" ...


In [19]:
### Tidying up the data again (Cleaning)
tide[c('PreTide', 'PostTide')] <- str_split_fixed(tide$PreTide_PostTide, '_', 2)
str(tide)

'data.frame':	90 obs. of  7 variables:
 $ Date            : int  20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 ...
 $ Moon            : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time            : num  0 0 0 4 4 4 8 8 8 9 ...
 $ Tide            : num  0.271 0.271 0.271 0.505 0.505 ...
 $ PreTide_PostTide: chr  "0.24_0.54" "0.24_0.54" "0.24_0.54" "0.54_-0.15" ...
 $ PreTide         : chr  "0.24" "0.24" "0.24" "0.54" ...
 $ PostTide        : chr  "0.54" "0.54" "0.54" "-0.15" ...


In [20]:
### Removing extra joint column (Cleaning)
tide_2 <- tide[c("Date", "Moon", "Time", "Tide", "PreTide", "PostTide")]
str(tide_2)

'data.frame':	90 obs. of  6 variables:
 $ Date    : int  20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 ...
 $ Moon    : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time    : num  0 0 0 4 4 4 8 8 8 9 ...
 $ Tide    : num  0.271 0.271 0.271 0.505 0.505 ...
 $ PreTide : chr  "0.24" "0.24" "0.24" "0.54" ...
 $ PostTide: chr  "0.54" "0.54" "0.54" "-0.15" ...


In [21]:
### Convert Tide column from inches to meters
tide_2$Tide <- tide_2$Tide * 0.3048
str(tide_2)

'data.frame':	90 obs. of  6 variables:
 $ Date    : int  20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 ...
 $ Moon    : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time    : num  0 0 0 4 4 4 8 8 8 9 ...
 $ Tide    : num  0.0826 0.0826 0.0826 0.1541 0.1541 ...
 $ PreTide : chr  "0.24" "0.24" "0.24" "0.54" ...
 $ PostTide: chr  "0.54" "0.54" "0.54" "-0.15" ...


In [22]:
### Changing PreTide and PostTide columns from character to double strings
tide_2$PreTide = as.double(tide_2$PreTide)
tide_2$PostTide = as.double(tide_2$PostTide)
str(tide_2)

'data.frame':	90 obs. of  6 variables:
 $ Date    : int  20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 ...
 $ Moon    : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time    : num  0 0 0 4 4 4 8 8 8 9 ...
 $ Tide    : num  0.0826 0.0826 0.0826 0.1541 0.1541 ...
 $ PreTide : num  0.24 0.24 0.24 0.54 0.54 0.54 0.54 0.54 0.54 -0.15 ...
 $ PostTide: num  0.54 0.54 0.54 -0.15 -0.15 -0.15 -0.15 -0.15 -0.15 2.28 ...


In [23]:
### Convert PreTide and PostTide columns from inches to meters
tide_2$PreTide <- tide_2$PreTide * 0.3048
tide_2$PostTide <- tide_2$PostTide * 0.3048
str(tide_2)

'data.frame':	90 obs. of  6 variables:
 $ Date    : int  20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 20220628 ...
 $ Moon    : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Time    : num  0 0 0 4 4 4 8 8 8 9 ...
 $ Tide    : num  0.0826 0.0826 0.0826 0.1541 0.1541 ...
 $ PreTide : num  0.0732 0.0732 0.0732 0.1646 0.1646 ...
 $ PostTide: num  0.1646 0.1646 0.1646 -0.0457 -0.0457 ...


In [24]:
#combine [UVF] and pple data with tidal data from tide_2
Diamond_all <- cbind(conc_pple_fin,tide_2)
head(Diamond_all)

,Moon,Date,Time,BP3,OC,People_total,ppl_msq,Date,Moon,Time,Tide,PreTide,PostTide
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Muku,20220628,0,12.34324,0,0,0,20220628,Muku,0,0.08264164,0.073152,0.164592
2,Muku,20220628,0,13.92103,0,0,0,20220628,Muku,0,0.08264164,0.073152,0.164592
3,Muku,20220628,0,11.11400,0,0,0,20220628,Muku,0,0.08264164,0.073152,0.164592
4,Muku,20220628,4,20.03424,0,0,0,20220628,Muku,4,0.15407579,0.164592,-0.045720
5,Muku,20220628,4,20.05967,0,0,0,20220628,Muku,4,0.15407579,0.164592,-0.045720
6,Muku,20220628,4,18.89155,0,0,0,20220628,Muku,4,0.15407579,0.164592,-0.045720


In [25]:
#remove duplicate columns
Diamond_all_1 <- Diamond_all[,-10]
Diamond_all_2 <- Diamond_all_1[,-9]
Diamond_all_fin <- Diamond_all_2[,-8]

In [26]:
head(Diamond_all_fin)
str(Diamond_all_fin)

,Moon,Date,Time,BP3,OC,People_total,ppl_msq,Tide,PreTide,PostTide
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Muku,20220628,0,12.34324,0,0,0,0.08264164,0.073152,0.164592
2,Muku,20220628,0,13.92103,0,0,0,0.08264164,0.073152,0.164592
3,Muku,20220628,0,11.11400,0,0,0,0.08264164,0.073152,0.164592
4,Muku,20220628,4,20.03424,0,0,0,0.15407579,0.164592,-0.045720
5,Muku,20220628,4,20.05967,0,0,0,0.15407579,0.164592,-0.045720
6,Muku,20220628,4,18.89155,0,0,0,0.15407579,0.164592,-0.045720


'data.frame':	90 obs. of  10 variables:
 $ Moon        : chr  "Muku" "Muku" "Muku" "Muku" ...
 $ Date        : num  20220628 20220628 20220628 20220628 20220628 ...
 $ Time        : int  0 0 0 4 4 4 8 8 8 9 ...
 $ BP3         : num  12.3 13.9 11.1 20 20.1 ...
 $ OC          : num  0 0 0 0 0 ...
 $ People_total: num  0 0 0 0 0 0 9 9 9 11 ...
 $ ppl_msq     : num  0 0 0 0 0 ...
 $ Tide        : num  0.0826 0.0826 0.0826 0.1541 0.1541 ...
 $ PreTide     : num  0.0732 0.0732 0.0732 0.1646 0.1646 ...
 $ PostTide    : num  0.1646 0.1646 0.1646 -0.0457 -0.0457 ...


In [27]:
#Export tidy csv file
write.csv(Diamond_all_fin, "DiamondBeach_all.csv")

In [28]:
#This is the start of mixed model creation

In [29]:
Diamond_all_fin$ppl_msq_s <- scale(Diamond_all_fin$ppl_msq) 

In [30]:
Diamond_all_fin$Time_s <- scale(Diamond_all_fin$Time) 

In [31]:
Diamond_all_fin$Tide_s <- scale(Diamond_all_fin$Tide) 

In [32]:
Diamond_all_fin$Time <- as.numeric(Diamond_all_fin$Time)

In [33]:
Diamond_all_fin = Diamond_all_fin %>%
mutate(Moon_s = recode(Diamond_all_fin$Moon, 
                      "Muku" = 1, 
                      "Hoku" = 2, 
                      "ʻOlekūkolu" = 3))

In [34]:
#Time of day effect on people m^2 
#P value shows effect on ppl_msq from time is significant
T_o_D_ppl_msq <- lm(ppl_msq_s ~ Time_s, data = Diamond_all_fin)
summary(T_o_D_ppl_msq)


Call:
lm(formula = ppl_msq_s ~ Time_s, data = Diamond_all_fin)

Residuals:
    Min      1Q  Median      3Q     Max 
-1.3323 -0.7431 -0.2025  0.8924  1.6842 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)  
(Intercept) 2.965e-17  1.037e-01   0.000   1.0000  
Time_s      2.075e-01  1.043e-01   1.989   0.0498 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.9838 on 88 degrees of freedom
Multiple R-squared:  0.04304,	Adjusted R-squared:  0.03216 
F-statistic: 3.958 on 1 and 88 DF,  p-value: 0.04977


In [35]:
# Concentration of OC by time of day and ppl_msq
Time.vs.OC <- lmer(OC ~ Time_s + ppl_msq_s + (1|Moon_s), data = Diamond_all_fin)
summary(Time.vs.OC)

ERROR: Error in lmer(OC ~ Time_s + ppl_msq_s + (1 | Moon_s), data = Diamond_all_fin): could not find function "lmer"


In [36]:
# Concentration of BP3 by time of day and ppl_msq
Time.vs.BP3 <- lmer(BP3 ~ Time_s + ppl_msq_s + (1|Moon_s), data = Diamond_all_fin)
summary(Time.vs.BP3)

ERROR: Error in lmer(BP3 ~ Time_s + ppl_msq_s + (1 | Moon_s), data = Diamond_all_fin): could not find function "lmer"


In [37]:
# Concentration of BP3 off tide, time, and ppl_msq
Tide.vs.BP3 <- lmer(BP3 ~ Tide_s + Time_s + ppl_msq_s + (1|Moon_s), data = Diamond_all_fin)
summary(Tide.vs.BP3)

ERROR: Error in lmer(BP3 ~ Tide_s + Time_s + ppl_msq_s + (1 | Moon_s), data = Diamond_all_fin): could not find function "lmer"


In [38]:
# Concentration of OC off tide, time, and ppl_msq
Tide.vs.OC <- lmer(OC ~ Tide_s + Time_s + ppl_msq_s + (1|Moon_s), data = Diamond_all_fin)
summary(Tide.vs.OC)

ERROR: Error in lmer(OC ~ Tide_s + Time_s + ppl_msq_s + (1 | Moon_s), data = Diamond_all_fin): could not find function "lmer"


In [39]:
# Concentration of OC versus ppl_msq and time
PPL.vs.OC <- lmer(OC ~ ppl_msq + Time_s + (1|Moon_s), data = Diamond_all_fin)
summary(PPL.vs.OC)

ERROR: Error in lmer(OC ~ ppl_msq + Time_s + (1 | Moon_s), data = Diamond_all_fin): could not find function "lmer"


In [51]:
# Concentration of BP3 versus ppl_msq and time
PPL.vs.BP3 <- lmer(BP3 ~ ppl_msq + Time_s + (1|Moon_s), data = Diamond_all_fin)
summary(PPL.vs.BP3)

ERROR: Error in lmer(BP3 ~ ppl_msq + Time_s + (1 | Moon_s), data = Diamond_all_fin): could not find function "lmer"


In [41]:
##BP3: effect of pretide(w/ time of day interaction) 
##and posttide (with tide of day interaction) on [BP3].
names(Diamond_all_fin)

[1] "Moon"         "Date"         "Time"         "BP3"          "OC"          
 [6] "People_total" "ppl_msq"      "Tide"         "PreTide"      "PostTide"    
[11] "ppl_msq_s"    "Time_s"       "Tide_s"       "Moon_s"

In [50]:
BP3_pre_post <- lmer(BP3 ~ PreTide*Time_s + PostTide*Time_s + (1|Moon_s), data = Diamond_all_fin)
summary(BP3_pre_post)

ERROR: Error in lmer(BP3 ~ PreTide * Time_s + PostTide * Time_s + (1 | Moon_s), : could not find function "lmer"
